In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE273630"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE273630"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE273630.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE273630.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE273630.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dopamine-regulated biomarkers in peripheral blood of HIV+ Methamphetamine users"
!Series_summary	"HIV and Methamphetamine study - Translational Methamphetamine AIDS Research Center - Dopamine-regulated inflammatory biomarkers"
!Series_summary	"A digital transcript panel was custom-made based on Hs_NeuroPath_v1 (Nanostring) to accommodate dopamine-regulated inflammatory genes that were previously identified in vitro, and hypothesized to cluster HIV+ Methamphetamine users."
!Series_overall_design	"Specimens were peripheral blood leukocytes isolated from participants that included adults enrolled by NIH-funded studies at the University of California San Diego’s HIV Neurobehavioral Research Program (HNRP) and Translational Methamphetamine Research Center (TMARC) under informed consent and approved protocols. The subset of PWH and PWoH selected for this study were by design males, between 35 – 44 years old, due to cohort characteristics and to increase

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Analyze gene expression data availability
is_gene_available = True  # Based on the background, this is a gene expression panel

# 2.1 Data Availability Analysis
# This dataset appears to study HIV+ methamphetamine users
# From the background information, we can infer:
# - The trait is Substance Use Disorder (methamphetamine use/dependence)
# - Age is controlled (all participants are 35-44 years old)
# - Gender is controlled (all participants are males)

# Since the sample characteristics don't show trait/age/gender explicitly,
# but the background information mentions HIV+/- and METH+/- groups,
# we need to infer these values from the overall design.

# The key for trait (METH use) doesn't exist in sample characteristics
trait_row = None  # Not directly available in the sample characteristics

# Age and gender are controlled variables (all males, 35-44 years)
age_row = None    # Not available as a variable (all subjects are 35-44 years)
gender_row = None # Not available as a variable (all subjects are males)

# 2.2 Data Type Conversion Functions
# Although we don't have actual data for these variables, we'll define 
# conversion functions that would be used if the data were available

def convert_trait(value):
    """Convert methamphetamine use status to binary."""
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'meth+' in value or 'methamphetamine+' in value or 'yes' in value:
        return 1
    elif 'meth-' in value or 'methamphetamine-' in value or 'no' in value:
        return 0
    return None

def convert_age(value):
    """Convert age to continuous value."""
    if value is None:
        return None
    value = str(value)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        age = float(value)
        return age
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'male' in value or 'm' == value:
        return 1
    elif 'female' in value or 'f' == value:
        return 0
    return None

# 3. Save metadata
# Trait data is not available as a variable in the sample characteristics
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 61
Header line: "ID_REF"	"GSM8434091"	"GSM8434092"	"GSM8434093"	"GSM8434094"	"GSM8434095"	"GSM8434096"	"GSM8434097"	"GSM8434098"	"GSM8434099"	"GSM8434100"	"GSM8434101"	"GSM8434102"	"GSM8434103"	"GSM8434104"	"GSM8434105"	"GSM8434106"	"GSM8434107"	"GSM8434108"	"GSM8434109"	"GSM8434110"	"GSM8434111"	"GSM8434112"	"GSM8434113"	"GSM8434114"	"GSM8434115"	"GSM8434116"	"GSM8434117"	"GSM8434118"	"GSM8434119"	"GSM8434120"	"GSM8434121"	"GSM8434122"	"GSM8434123"	"GSM8434124"	"GSM8434125"	"GSM8434126"	"GSM8434127"	"GSM8434128"	"GSM8434129"	"GSM8434130"	"GSM8434131"	"GSM8434132"	"GSM8434133"	"GSM8434134"	"GSM8434135"	"GSM8434136"	"GSM8434137"	"GSM8434138"	"GSM8434139"	"GSM8434140"	"GSM8434141"	"GSM8434142"	"GSM8434143"	"GSM8434144"	"GSM8434145"	"GSM8434146"	"GSM8434147"	"GSM8434148"	"GSM8434149"	"GSM8434150"	"GSM8434151"	"GSM8434152"	"GSM8434153"	"GSM8434154"	"GSM8434155"	"GSM8434156"	"GSM8434157"	"GSM8434158"	"GSM8434159"	"GSM8434160"	"GSM8434161"	"GSM8434162"	"GSM8434163"	

### Step 4: Gene Identifier Review

In [5]:
# I need to analyze the gene identifiers in the data
# Looking at the first data line, I see identifiers like "ABAT", "ABL1", "ACAA1", "ACHE", etc.
# These appear to be standard human gene symbols rather than probe IDs or other identifiers

# Based on the sample data, I can see that the identifiers are already in the form of standard 
# human gene symbols (e.g., ABAT, ABL1, ACAA1, etc.). These are the official gene symbols 
# used in human genomics and do not require mapping.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (780, 99)
Gene data shape after normalization: (780, 99)
Normalized gene data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE273630.csv
Trait row is None. Cannot extract trait information from clinical data.
Abnormality detected in the cohort: GSE273630. Preprocessing failed.
Data was determined to be unusable due to missing trait indicators and was not saved
